#### Calculando o Beta de uma Ação

In [1]:
# Importanto bibliotecas necessárias:
import numpy as np
import pandas as pd
from pandas_datareader import data as wb

In [2]:
# Carregando os dados:
tickers = ['PG','^GSPC']
data = pd.DataFrame()
for t in tickers:
    data[t] = wb.DataReader(t,data_source='yahoo', start='2017-10-17', end='2022-9-21')['Adj Close']

data.tail()

,PG,^GSPC
Date,,
2022-09-15,137.440002,3901.350098
2022-09-16,138.279999,3873.330078
2022-09-19,137.990005,3899.889893
2022-09-20,137.500000,3855.929932
2022-09-21,135.729996,3789.929932


In [3]:
# Calculando os retornos:
sec_returns = np.log(data / data.shift(1))

sec_returns.tail()

,PG,^GSPC
Date,,
2022-09-15,-0.007755,-0.011382
2022-09-16,0.006093,-0.007208
2022-09-19,-0.002099,0.006834
2022-09-20,-0.003557,-0.011336
2022-09-21,-0.012956,-0.017265


In [4]:
# Calculando a Matriz de Covariância dos Retornos Anualizada:
cov = sec_returns.cov() * 250

cov

,PG,^GSPC
PG,0.046792,0.026302
^GSPC,0.026302,0.045307


In [5]:
# Extraindo a covariância entre a ação e a carteira de mercado:
cov_with_market = cov.iloc[0,1] # segunda celula da matriz de covariancia

cov_with_market

0.02630234016862865

In [6]:
# Calculando a Variância da Carteira de Mercado anualizada:
market_var = sec_returns['^GSPC'].var() * 250

market_var

0.04530676220096192

In [7]:
# Calculando o Beta:
PG_beta = cov_with_market / market_var

print(f'Beta da PG: {PG_beta:.2f}')

Beta da PG: 0.58


In [8]:
def tipo_acao(beta):
    if beta == 0:
        return f'{beta:.2f}: Sem relação com o mercado.'
    elif beta < 1:
        return f'{beta:.2f}: Ação Defensiva.'
    elif beta == 1:
        return f'{beta:.2f}: Risco igual ao do mercado.'
    elif beta > 1:
        return f'{beta:.2f}: Ação Arriscada/Volátil.'
    
print(tipo_acao(PG_beta))

0.58: Ação Defensiva.


#### Calculando o Retorno Esperado de uma Ação - CAPM

In [9]:
rf = 2.5 / 100 # retorno dos titulos americanos de 10 anos
rm_rf = 5 / 100 # premio de risco de mercado pada ações nos EUA

# Calculo do Retorno Esperado
PG_er = rf + PG_beta * rm_rf

print(f'Retorno Esperado da Ação da P&G: {PG_er * 100:.2f}%')

Retorno Esperado da Ação da P&G: 5.40%


#### Calculando o Índice de Sharpe

In [11]:
rf = 2.5 / 100

Sharpe = (PG_er - rf) / (sec_returns['PG'].std() * 250 ** 0.5)
# PG_er - retorno esperado do ativo calculado pelo CAPM
# rf - retorno livre de risco utilizado no mercado
# sec_returns['PG'].std() * 250 ** 0.5 - desvio-padrão dos retornos do ativo anualizado - risco do ativo

print(f'Índice Sharpe da PG: {Sharpe:.2f}')

Índice Sharpe da PG: 0.13
